# Processing FISH images

In [1]:
from pathlib import Path
from ipyfilechooser import FileChooser
fc = FileChooser()
display(fc)


FileChooser(path='/home/jeromeb/work/code/octopus', filename='', title='', show_hidden=False, select_desc='Sel…

In [ ]:
path = Path(fc.selected)
folder = path.parent
destdir = Path('./')
#destdir = folder
filename = path.name

In [4]:
srcdir = Path('/run/user/1000/gvfs/smb-share:server=cephfs2,share=users/acourtney/HCR_Octopus_Jerome/Febuary2024/')
filename = '14Aug2023_3b_Slices_F1_Stitched_Cropped_ForAnalysis.ims'
destdir = Path('./')


In [79]:
import numpy as np
import dask.array as da
from skimage import measure
import pandas as pd
import json
from imaris_ims_file_reader.ims import ims
import tifffile
import napari
from cellpose import models, core
from scipy import ndimage as ndi

model = models.Cellpose(gpu=core.use_gpu(), model_type="nuclei")

resolution_level = 3 


cached file /home/jeromeb/.cellpose/models/nucleitorch_0
url :https://www.cellpose.org/models/nucleitorch_0
cached file /home/jeromeb/.cellpose/models/size_nucleitorch_0.npy
url :https://www.cellpose.org/models/size_nucleitorch_0.npy


In [78]:
filelist = pd.DataFrame({'folder': [p.parent for p in folder.glob('*.ims')],'name': [p.name for p in folder.glob('*.ims')]})
filelist.to_csv(destdir/'filelist.csv')

In [81]:
def load_regions_as_labels(row, destdir, shape):
   
    with open(destdir/f'{path.stem}-regions.json', "r") as outfile:
        poly = json.load(outfile)
    
    c = resolution_level / poly['resolution_level']    
    sdata = [c * np.array(p) for p in poly['regions']]
    rois2d = napari.layers.Shapes(sdata, shape_type='polygon').to_labels(labels_shape=[shape[1],shape[2]])
    rois = np.zeros(shape)
    for k in range(shape):
        rois[k,0:rois2d.shape[0],0:rois2d.shape[1]] = rois2d
    return rois

def preprocess(img):
    d = ndi.gaussian_filter(img,1) - ndi.gaussian_filter(img,3)    
    t = 2 * 1.48 * np.median(np.abs(d - np.median(d)))
    return np.maximum(0, d - t)

def process_item(row, resolution_level, model, dstdir):    
    """Process a line of the input filelist
    Segment the nuclei and measure the FISH signal
    """
    # load the image
    store = ims(folder / filename, ResolutionLevelLock=resolution_level, aszarr=True)
    img = da.from_zarr(store)
    shape = img.shape

    # segment the cell
    labels = model.eval(
            img[:,3].compute().squeeze(),
            diameter=30/resolution_level,
            do_3D=False,
            anisotropy=3,        
            min_size=500,
            stitch_threshold=0.1,        
        )[0]
    tifffile.imwrite(dstdir / row['name'].replace('.ims','-labels.tif'), labels)

    # load the regions
    rois = load_regions_as_labels(row, destdir, shape)
    
    score = np.stack([preprocess(img[0,k].compute().squeeze().astype(np.float32)) for k in range(3)],-1)
    
    props1 = measure.regionprops(labels, score)
    props2 = measure.regionprops(labels, rois)
    import pandas as pd
    df = pd.DataFrame([ {
        'label': p.label,
        'ch1': p.intensity_mean[0] * p.area,
        'ch2': p.intensity_mean[1] * p.area,
        'ch3': p.intensity_mean[2] * p.area,
        'x': p.centroid[2],
        'y': p.centroid[1],
        'z': p.centroid[0],
        'roi': int(q.intensity_min),
        } for p, q in zip(props1, props2)])

    df.to_csv(dstdir / row['name'].replace('.ims','-measurements.csv'))

    return df

In [ ]:
for row in filelist.iloc:
    process_item(row, resolution_level, model, destdir)